In [0]:
%sql
CREATE DATABASE IF NOT EXISTS demo;

In [0]:
SHOW DATABASES; 

In [0]:
DESCRIBE DATABASE demo; 

In [0]:
SELECT current_database()

In [0]:
USE demo; -- set this table into default

In [0]:
SHOW TABLES; 

In [0]:
SHOW TABLES IN DEFAULT; 

# CREATE MANAGED TABLE USING PYTHON

In [0]:
%run "../includes/configuration"

In [0]:
%python 
race_results_df = spark.read.parquet(f"{PRESENTATION_FOLDER_PATH}/race_results")

In [0]:
%python 
race_results_df.write.format("parquet").saveAsTable("demo.race_results_python")

In [0]:
DESC EXTENDED demo.race_results_python -- describe the table 

# WORKING WITH EXTERNAL TABLE

In [0]:
%python 
race_results_df.write.format("parquet").options("path", f"{PRESENTATION_FOLDER_PATH}/race_results_ext_py"). \ # specify the path where to save the data
    saveAsTable(f"{PRESENTATION_FOLDER_PATH}/demo.race_results_ext_py")

In [0]:
DESC EXTENDED demo.race_results_ext_py -- describe the table 

# CREATE EXTERNAL TABLE USING SQL

In [0]:
CREATE TABLE demo.race_results_ext_sql 
(race_year INT,
race_name STRING,
race_date TIMESTAMP,
circuit_location STRING,
driver_name STRING,
driver_number INT,
driver_nationality STRING,
team STRING,
grid INT,
fastest_lap INT, 
race_time STRING,
points FLOAT, 
position INT,
created_date TIMESTAMP
)
USING PARQUET
LOCATION "/mnt/formula1in2025/presentation/race_results_ext_sql"

In [0]:
-- you might want to ingest data from another, NOT adding row 1 by 1 
INSERT INTO demo.race_results_ext_sql 
SELECT * FROM demo.race_results_ext_py WHERE race_year = 2020

# VIEWS IN SQL

## create LOCAL temporary view

In [0]:
CREATE OR REPLACE TEMP VIEW demo.lv_race_results
AS 
  (
    SELECT * 
    FROM demo.race_results_python
    WHERE race_year = 2020
  )

## create GLOBAL temporary view

In [0]:
CREATE OR REPLACE GLOBAL TEMP VIEW demo.gv_race_results
AS 
  (
    SELECT * 
    FROM demo.race_results_python
    WHERE race_year = 2020
  )

In [0]:
SELECT * FROM global_temp.gv_race_results

## create permanent view

In [0]:
CREATE OR REPLACE VIEW demo.pv_race_results
AS 
  (
    SELECT * 
    FROM demo.race_results_python
    WHERE race_year = 2020
  )

In [0]:
SHOW TABLES

### Some useful functions in SQL

To concat columns, use `concat`:

```
SELECT *, CONCAT(driver_ref, "-", code) AS new_driver_ref
FROM drivers
```

To split the value of a column, use `split`. For example, this query will result in a new column containing arrays, with each array has values that are split (by ' ') in the query:
```
SELECT *, SPLIT(name, ' ')
FROM drivers
```

In case we want to clarify the split by assigning the values, we can query like this:
```
SELECT *, SPLIT(name, ' ')[0] forename, SPLIT(name, ' ')[1] surname
FROM drivers
```


To calculate the average of the whole dataset and (maybe) a part of the dataset based on some criteria, we use: 

```sql
SELECT
    car_make,
    car_model,
    car_price,
    AVG(car_price) OVER() AS "overall average price", -- calculate the average price across all cars in the price list
    AVG(car_price) OVER (PARTITION BY car_type) AS "car type average price" -- calculate the average price of a specific car_type
FROM car_list_prices

```

### Explanation about JOINS

Suppose there are 2 tables TableA and TableB with only 2 columns (Id, Data) and following data:
```
TableA:
+----+---------+
| Id |  Data   |
+----+---------+
|  1 | DataA11 |
|  1 | DataA12 |
|  1 | DataA13 |
|  2 | DataA21 |
|  3 | DataA31 |
+----+---------+


TableB:

+----+---------+
| Id |  Data   |
+----+---------+
|  1 | DataB11 |
|  2 | DataB21 |
|  2 | DataB22 |
|  2 | DataB23 |
|  4 | DataB41 |
+----+---------+
```

- Inner Join on column Id will return columns from both the tables and only the matching records:

```
.----.---------.----.---------.
| Id |  Data   | Id |  Data   |
:----+---------+----+---------:
|  1 | DataA11 |  1 | DataB11 |
:----+---------+----+---------:
|  1 | DataA12 |  1 | DataB11 |
:----+---------+----+---------:
|  1 | DataA13 |  1 | DataB11 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB21 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB22 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB23 |
'----'---------'----'---------'

```

- Left Join (or Left Outer join) on column Id will return columns from both the tables and matching records with records from left table (Null values from right table):
```
.----.---------.----.---------.
| Id |  Data   | Id |  Data   |
:----+---------+----+---------:
|  1 | DataA11 |  1 | DataB11 |
:----+---------+----+---------:
|  1 | DataA12 |  1 | DataB11 |
:----+---------+----+---------:
|  1 | DataA13 |  1 | DataB11 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB21 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB22 |
:----+---------+----+---------:
|  2 | DataA21 |  2 | DataB23 |
:----+---------+----+---------:
|  3 | DataA31 |    |         |
'----'---------'----'---------'
```

- Right Join (or Right Outer join) on column Id will return columns from both the tables and matching records with records from right table (Null values from left table):
```
┌────┬─────────┬────┬─────────┐
│ Id │  Data   │ Id │  Data   │
├────┼─────────┼────┼─────────┤
│  1 │ DataA11 │  1 │ DataB11 │
│  1 │ DataA12 │  1 │ DataB11 │
│  1 │ DataA13 │  1 │ DataB11 │
│  2 │ DataA21 │  2 │ DataB21 │
│  2 │ DataA21 │  2 │ DataB22 │
│  2 │ DataA21 │  2 │ DataB23 │
│    │         │  4 │ DataB41 │
└────┴─────────┴────┴─────────┘
```

- Full Outer Join on column Id will return columns from both the tables and matching records with records from left table (Null values from right table) and records from right table (Null values from left table):
```
╔════╦═════════╦════╦═════════╗
║ Id ║  Data   ║ Id ║  Data   ║
╠════╬═════════╬════╬═════════╣
║  - ║         ║    ║         ║
║  1 ║ DataA11 ║  1 ║ DataB11 ║
║  1 ║ DataA12 ║  1 ║ DataB11 ║
║  1 ║ DataA13 ║  1 ║ DataB11 ║
║  2 ║ DataA21 ║  2 ║ DataB21 ║
║  2 ║ DataA21 ║  2 ║ DataB22 ║
║  2 ║ DataA21 ║  2 ║ DataB23 ║
║  3 ║ DataA31 ║    ║         ║
║    ║         ║  4 ║ DataB41 ║
╚════╩═════════╩════╩═════════╝
```

- Left Semi Join on column Id will return columns only from left table and matching records only from left table. If there are multiple matching rows in the right-hand column, it will only return the rows from the left table, regardless of the number of matching rows on the right side.
```
┌────┬─────────┐
│ Id │  Data   │
├────┼─────────┤
│  1 │ DataA11 │
│  1 │ DataA12 │
│  1 │ DataA13 │
│  2 │ DataA21 │
└────┴─────────┘

```